<a href="https://colab.research.google.com/github/TheCaduceus/Multi-Cloud-Transfer-Tool/blob/main/Rclone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**First Connect your Google Account! So that Rclone can save/fetch its Progress**

In [ ]:
#@markdown <br><center><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png" height="50" alt="Gdrive-logo' "/></center>
#@markdown <center><h3>Connect to Google Account</h3></center><br>
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
#Mount your Gdrive! 
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

**First Download Required Files by Running the Below Code:**

In [ ]:
#@markdown <br><center><img src="https://www.caduceus.ml/Img/Setup_img.png" height="50" alt="Setup-Logo' "/></center>
#@markdown <center><h3><b>Setup & Configure Environment</b></h3></center><br>
Stable = ""
Beta = "-s beta"
Install_Rclone_Flavour = Stable #@param ["Stable", "Beta"] {type:"raw"}
print("Successfully Connected to Runtime!")
print("Download Request Initiated")
! curl https://www.caduceus.ml/files/install.sh | sudo bash {Install_Rclone_Flavour}

<b>Fetch your Rclone Config File! (If any) from your Google Drive's My Drive, if you want to continue your work with previously created Remotes!<font color="red"> Please Avoid storing too many Rclone Config file in your My Drive.</font>(Optional)</b>

In [ ]:
#@markdown <br><center><img src="https://1.bp.blogspot.com/-M5PLcSana6M/XgBHF7jUjiI/AAAAAAAAUzs/S24qhuijluwKlzIOnc2gntoI-U83ZsrJACLcBGAsYHQ/s1600/rclone_logo.png" height="50" alt="Rclone-Logo' "/></center>
#@markdown <center><h3><b>Fetch Previous Rclone Config File from My Drive</b></h3></center><br>
!rclone delete /root/.config/rclone/
print("Checking for duplicates!")
!rclone copy /content/drive/MyDrive/rclone.conf /root/.config/rclone/
print("Successfully Fetched!")

**Using Service Accounts: Just zip the "accounts" folder which have 100 json files and upload it using the below code. Name of zip should be ``accounts.zip``. Then copy the path of json file and use it to create Rclone Config file below. (Optional)** ***(Refer Guide for detailed help)***

In [ ]:
#@markdown <br><center><img src="https://pluspng.com/img-png/google-cloud-logo-png-google-cloud-logo-evolution-history-and-meaning-png-1280x720.png" height="150" alt="Rclone-Logo' "/></center>
#@markdown <center><h3><b>Upload accounts.zip and use Service Accounts</b></h3></center><br>
from google.colab import files
from google.colab import output
# /content/accounts.zip
print("Upload accounts.zip")
files.upload()
!unzip "/content/accounts.zip" -d "/content/"
output.clear()
print("Successfully Received & Unzipped the file!")

**Lets Create/Edit/View/Delete your Rclone Config file:<br>``Upload_Config_File``: Upload your already made Rclone config file to use it.<br>``Save_Config_MyDrive``: To save created Rclone Config File in Google Drive's My Drive.<br>``Download_Config``: Download Created Rclone Config file in your Device.**


In [ ]:
#@markdown <br><center><img src="https://1.bp.blogspot.com/-M5PLcSana6M/XgBHF7jUjiI/AAAAAAAAUzs/S24qhuijluwKlzIOnc2gntoI-U83ZsrJACLcBGAsYHQ/s1600/rclone_logo.png" height="50" alt="Rclone-Logo' "/></center>
#@markdown <center><h3><b>Create/Edit/Delete Rclone Config File</b></h3></center><br>
from google.colab import files
from google.colab import output
import time
#@markdown ****
Upload_Config_File = False #@param {type:"boolean"} 
Save_Config_MyDrive = True #@param {type:"boolean"}
Download_Config = False #@param {type:"boolean"}
print("Script by Dr.Caduceus")
if(Upload_Config_File==True):
  !rclone delete "/root/.config/rclone/rclone.conf"
  output.clear()
  config_file = files.upload()
  !rclone move "/content/rclone.conf" "/root/.config/rclone/"
  print("Reading Config File")
  print(config_file)
  time.sleep(2)
if(Save_Config_MyDrive + Download_Config==False):
  print("Warning! You have not selected any saving options for Rclone config file. It will be automatically deleted once this Runtime disconnect.")
!rclone config
if(Save_Config_MyDrive==True):
  print("Getting Config File!")
  print("Uploading Rclone config to Google Drive (My Drive)...")
  !rclone copy "/root/.config/rclone/rclone.conf" "/content/drive/MyDrive/"
  print("Rclone config saved to your Google Drive in My Drive!")
if(Download_Config==True):
  print("Getting Config File!")
  print("Initiating Download Request to your Browser.")
  files.download("/root/.config/rclone/rclone.conf")

**Transfer Tools:**

**Connect your Cloud Service Accounts, between which you want to transfer your Data! Run the Following Code. It will show the names of configured Cloud Accounts by you in your Rclone Config file.**

In [ ]:
#@markdown <br>
#@markdown <center><h3><b>Connect To Cloud Accounts</b></h3></center>
#@markdown <br><center><img src="https://cdn1.iconfinder.com/data/icons/cloud-system-color-line/64/cluster-512.png" height="100" alt="Rclone-Logo' "/></center>
#@markdown <center><b>Connect your Cloud Accounts (Through Rclone Config file!) between which you want to transfer Data!</b></center>
import os
from IPython.display import HTML, clear_output
import uuid
import ipywidgets as widgets
from google.colab import output
import re
##########################################

class MakeButton(object):
  def __init__(self, title, callback, style):
    self._title = title
    self._callback = callback
    self._style = style
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    if self._style != "":
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button mod-" + self._style
    else:
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button"
    template = """<button class="{style_html}" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id, style_html=style_html)
    return html
  
def ShowAC():
  clear_output(wait=True)
  display(
      widgets.HBox(
          [widgets.VBox(
              [widgets.HTML(
                  '''<h3 style="font-family:Trebuchet MS;color:#4f8bd6;margin-top:0px;">
                  Rclone available config...</h3>
                  '''
                  ),
               mountNam]
               )
          ]
          )
      )
  
  display(HTML("<br>"), MakeButton("Mount", MountCMD, "primary"),
          MakeButton("Unmount", unmountCMD, "danger"))
content = open("/root/.config/rclone/rclone.conf").read()
avCon = re.findall(r"^\[(.+)\]$", content, re.M)
mountNam = widgets.Dropdown(options=avCon)
cache_path="/content/temp/rCloneTemp"
def MountCMD():
    mPoint = f"/content/drives/{mountNam.value}"
    os.makedirs(mPoint, exist_ok=True)
    !rclone mount $mountNam.value: $mPoint --user-agent 'Mozilla' --buffer-size 256M --transfers 10 --vfs-cache-mode minimal --vfs-read-chunk-size 500M --vfs-cache-max-size 50G --vfs-cache-max-age 0h0m1s --vfs-cache-poll-interval 0m1s --cache-dir '/content/temp/rCloneTemp' --allow-other --daemon 

    if os.path.isdir(mPoint)== True:
      print(f"Mount success! - \t{mPoint}")
    else:
      print(f"Mount failed! - \t{mPoint}")

def unmountCMD():
  mPoint = f"/content/drives/{mountNam.value}"
  if os.system(f"fusermount -uz {mPoint}") == 0:
    runSh(f"rm -r {mPoint}")
    print(f"Unmounted success! - \t{mPoint}")
  else:
    runSh(f"fusermount -uz {mPoint}", output=True)

ShowAC()

**Before Transferring your File or Folders! Run the "Time out Preventer" to prevent Google Colab from getting time out.**

In [ ]:
#@markdown <br><center><img src='https://miro.medium.com/max/2036/1*GW4cqNv-SIQ9BGGhssL1bQ.png' height="100" alt="netdata"/></center>
#@title <center><b>Time Out Preventer (Advanced) </b></center></strong>
%%capture
AUTO_RECONNECT = True #@param {type:"boolean"}
#@markdown **Run this code to prevent Google Colab from Timeout**
from os import makedirs
makedirs("/root/.config/rclone", exist_ok = True)
if AUTO_RECONNECT:
  import IPython
  from google.colab import output

  display(IPython.display.Javascript('''
  function ClickConnect(){
    btn = document.querySelector("colab-connect-button")
    if (btn != null){
      console.log("Click colab-connect-button"); 
      btn.click() 
      }
    
    btn = document.getElementById('ok')
    if (btn != null){
      console.log("Click reconnect"); 
      btn.click() 
      }
    }
    
  setInterval(ClickConnect,60000)
  '''))

**Now Enter the Source & Destination Address and Notification settings to get notify once your Task got Completed! To get both Address click on File Icon.**

In [ ]:
#@markdown <br>
#@markdown <center><img src="https://forum.rclone.org/uploads/default/original/2X/d/da6ccb2784ff3fa73d9339300530e0aae4d6cebd.png" alt="rclone forum" width=150></center>
 
#@markdown ---
source = "" #@param {type:"string"}
destination = "" #@param {type:"string"}
mode = "copy" #@param ["copy", "copyto", "copyurl", "move", "moveto", "check", "sync", "bisync", "cat", "mkdir","cleanup", "ls", "lsd", "lsl", "md5sum", "sha1sum", "size", "genautocomplete", "gendocs", "tree","obscure"]
dry_run = False #@param {type:"boolean"}
Extra_args = "" #@param {type:"string"}
#@markdown ---
Email_notification = False #@param {type:"boolean"}
logs = False #@param {type:"boolean"}
emailID = "" #@param {type:"string"}
password = "" #@param {type:"string"}
Receiver_ID = "" #@param {type:"string"}
#@markdown ---
import smtplib 
########################################
args = "--transfers 20 --checkers 20 --stats-one-line --stats=5s -v --tpslimit 95 --tpslimit-burst 40 "
if mode == "check":
  args += "--one-way "
if dry_run == True:
    args +=" --dry-run "
if logs == True:
  args += " --log-file rClone_log.txt "
if Extra_args != "":
  args +=Extra_args
######################################
def runrClone():
  !rclone --user-agent "Mozilla" "$mode" "$source" "$destination" $args
  
def checkEmail():
  if (Email_notification == True and logs == False):
    print("You will receive only a notification after task has finished.")
    runrClone()
    #Send only notification
     
    s = smtplib.SMTP('smtp.gmail.com', 587) 
    s.starttls() 
    s.login(emailID , password)
    message = "Your rClone task has Completed!"
    s.sendmail(emailID, Receiver_ID, message)
    print("Email Alert Sent!")  
    s.quit()
  elif (Email_notification == True and logs == True):
    print("You will receive a notification with log attached after task has finished.")
    runrClone()
    #Sending email notification with logs
    
    from email.mime.multipart import MIMEMultipart 
    from email.mime.text import MIMEText 
    from email.mime.base import MIMEBase 
    from email import encoders 

    fromaddr = emailID
    toaddr = Receiver_ID
    msg = MIMEMultipart() 
    # storing the senders email address 
    msg['From'] = fromaddr 
    # storing the receivers email address 
    msg['To'] = toaddr 
    # storing the subject 
    msg['Subject'] = "Colab has Finished Running your Cell"
    # string to store the body of the mail 
    body = "Your rClone Task has Completed!"

    msg.attach(MIMEText(body, 'plain'))
    filename = "rClone_log.txt"
    attachment = open("/content/rClone_log.txt", "rb")
    p = MIMEBase('application', 'octet-stream') 
    p.set_payload((attachment).read()) 
    # encode into base64 
    encoders.encode_base64(p) 
 
    p.add_header('Content-Disposition', "attachment; filename= %s" % filename) 
    msg.attach(p)  
    s = smtplib.SMTP('smtp.gmail.com', 587) 
    s.starttls() 
    s.login(fromaddr, password) 
    text = msg.as_string() 
    s.sendmail(fromaddr, toaddr, text)
    print("Email Alert Sent with log!")  
    s.quit() 
  elif (Email_notification == False and logs == False):
    print("You will not receive any notification!!!.")
    runrClone()
checkEmail()

**Delete your Files/Folder/Directory from below!<br>``Delete`` This Mode will delete the content of folder or single file.<br>``Purge`` To delete whole Directory and leaving nothing behind.<br>``rmdir`` Completely Delete the given path.<br>``rmdirs`` Remove any empty directories under the path.**

In [ ]:
import time
# User Input
Action = "delete" #@param ["delete", "purge", "rmdir", "rmdirs"]
Path = "" #@param {type:"string"} 
Arguments = "" #@param {type:"string"}
# Command
print("Action Selected:" + Action)
print("Selected Wrong Action? You have 10 Seconds to cancel this Task.")
time.sleep( 10 )
!rclone {Action} {Path} {Arguments}
# Output
print("Task Completed!")

###**Special Add-ons:**

**Time to combine the power of Rclone with various Engines to make your hard work into effective & joyful.**

**<font color="yellow">1.You can put Source and Destination between " " to avoid wrong token Error. (Optional)</font>**

In [ ]:
#LibTorrent Installation
import libtorrent as lt
ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []

# User Input
Source = "" #@param {type:"string"} 
Mode = "YouTube Videos/Playlist" #@param ["Direct Download Links", "Download Links from txt file", "YouTube Videos/Playlist", "Magnet Links", "Torrent Files", "Mirror GDTOT Links"]
Destination = "" #@param {type:"string"} 
Extra_args = "" #@param {type:"string"}
args = Extra_args 
# Task Completed Message
output = "Task Completed Successfully!"
# Work
if(Mode=="Direct Download Links"):
  %cd {Destination}
  !aria2c {Source} {args}
  print(output)
if(Mode=="Download Links from txt file"):
  %cd {Destination}
  !aria2c -i {Source} {args}
if(Mode=="YouTube Videos/Playlist"):
  %cd {Destination}
  print("Using yt-dlp, video will be downloaded in max quality.")
  !yt-dlp {Source} {args}
  print(output)
if(Mode=="Magnet Links"):
  params = {"save_path": str(Destination)}

  while True:
        magnet_link = input("Enter Magnet Link Or Type Exit: ")
        if magnet_link.lower() == "exit":
                break
        downloads.append(
                lt.add_magnet_uri(ses, magnet_link, params)
        )
  import time
  from IPython.display import display
  import ipywidgets as widgets

  state_str = [
        "queued",
        "checking",
        "downloading metadata",
        "downloading",
        "finished",
        "seeding",
        "allocating",
        "checking fastresume",
  ]

  layout = widgets.Layout(width="auto")
  style = {"description_width": "initial"}
  download_bars = [
        widgets.FloatSlider(
                step=0.01, disabled=True, layout=layout, style=style
        )
        for _ in downloads
  ]
  display(*download_bars)

  while downloads:
        next_shift = 0
        for index, download in enumerate(downloads[:]):
                bar = download_bars[index + next_shift]
                if not download.is_seed():
                        s = download.status()

                        bar.description = " ".join(
                                [
                                        download.name(),
                                        str(s.download_rate / 1000),
                                        "kB/s",
                                        state_str[s.state],
                                ]
                        )
                        bar.value = s.progress * 100
                else:
                        next_shift -= 1
                        ses.remove_torrent(download)
                        downloads.remove(download)
                        bar.close() # Not Working
                        download_bars.remove(bar)
                        print(download.name(), "complete")
        time.sleep(1)
if(Mode=="Torrent Files"):
  from google.colab import files

  source = files.upload()
  params = {
        "save_path": str(Destination),
        "ti": lt.torrent_info(list(source.keys())[0]),
  }
  downloads.append(ses.add_torrent(params))
  import time
  from IPython.display import display
  import ipywidgets as widgets

  state_str = [
        "queued",
        "checking",
        "downloading metadata",
        "downloading",
        "finished",
        "seeding",
        "allocating",
        "checking fastresume",
  ]

  layout = widgets.Layout(width="auto")
  style = {"description_width": "initial"}
  download_bars = [
        widgets.FloatSlider(
                step=0.01, disabled=True, layout=layout, style=style
        )
        for _ in downloads
  ]
  display(*download_bars)

  while downloads:
        next_shift = 0
        for index, download in enumerate(downloads[:]):
                bar = download_bars[index + next_shift]
                if not download.is_seed():
                        s = download.status()

                        bar.description = " ".join(
                                [
                                        download.name(),
                                        str(s.download_rate / 1000),
                                        "kB/s",
                                        state_str[s.state],
                                ]
                        )
                        bar.value = s.progress * 100
                else:
                        next_shift -= 1
                        ses.remove_torrent(download)
                        downloads.remove(download)
                        bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
                        download_bars.remove(bar)
                        print(download.name(), "complete")
        time.sleep(1)
if(Mode=="Mirror GDTOT Links"):
  import re
  import base64
  import requests
  from urllib.parse import urlparse, parse_qs

  URL = Source
  print("Enter GDTOT Crypt")
  crypt = input("")
# ==========================================
  print("Bypassing Link...")
  def parse_info(res):
        title = re.findall(">(.*?)<\/h5>", res.text)[0]
        info = re.findall('<td\salign="right">(.*?)<\/td>', res.text)
        parsed_info = {
                'error': True,
                'message': 'Link Invalid.',
                'title': title,
                'size': info[0],
                'date': info[1]
        }
        return parsed_info

# ==========================================

  def gdtot_dl(url):
        client = requests.Session()
        client.cookies.update({ 'crypt': crypt })
        res = client.get(url)

        info = parse_info(res)
        info['src_url'] = url

        res = client.get(f"https://new.gdtot.top/dld?id={url.split('/')[-1]}")
    
        try:
                url = re.findall('URL=(.*?)"', res.text)[0]
        except:
                info['message'] = 'The requested URL could not be retrieved.',
                return info

        params = parse_qs(urlparse(url).query)
    
        if 'msgx' in params:
                info['message'] = params['msgx'][0]
    
        if 'gd' not in params or not params['gd'] or params['gd'][0] == 'false':
                return info
    
        try:
                decoded_id = base64.b64decode(str(params['gd'][0])).decode('utf-8')
                gdrive_url = f'https://drive.google.com/open?id={decoded_id}'
                info['message'] = 'Success.'
        except:
                info['error'] = True
                return info

        info['gdrive_link'] = gdrive_url
    
        return info
    
# ==========================================

  info = gdtot_dl(URL)
  print("File Moved to My Drive")
  print(info)

###**Packing & Extraction:**

**<font color="yellow">1.Put Custom Name + Source Path + Destination between "" to avoid wrong token Error.</font><br><font color="red">2.Never put "/" at the end of Source_Path or Destination_Path.**

In [ ]:
from google.colab import output
# User Input
Action = "Zip" #@param ["Zip", "UnZip", "RaR", "RaR with Password", "UnRaR", "UnRaR with Password", "Tar", "UnTar", "Tar.gz", "UnTar.gz", "Tar.bz2", "UnTar.bz2", "Extract 7z"]
Packed_Output_File_Name = "\"Custom Name Here only while packing\"" #@param {type:"string"}
Source_Path = "\"Source Path Here\"" #@param {type:"string"}
Destination_Path = "\"Destination Path Here\"" #@param {type:"string"}
# Packing + Custom Name
Pack_Dest = Destination_Path + "/" + Packed_Output_File_Name
UnPack_Dest = Destination_Path
sz_Dest = Destination_Path + "/"
Tar_Dest = Destination_Path + "/" + Packed_Output_File_Name + ".tar"
Targz_Dest = Destination_Path + "/" + Packed_Output_File_Name + ".tar.gz"
Tarbz_Dest = Destination_Path + "/" + Packed_Output_File_Name + ".tar.bz"
# Task Completed Message
output = "Task Completed Successfully!"
# Process
if(Action=="Zip"):
  !zip {Pack_Dest} {Source_Path}
  print(output)
if(Action=="UnZip"):
  !unzip {Source_Path} -d {UnPack_Dest}
  print(output)
if(Action=="RaR"):
  !apt-get install rar
  !rar a {Pack_Dest} {Source_Path}
  print(output)
if(Action=="RaR with Password"):
  !apt-get install rar
  !rar a -p {Pack_Dest} {Source_Path}
  print(output)
if(Action=="UnRaR"):
  !unrar x {Source_Path} {UnPack_Dest}
  print(output)
if(Action=="UnRaR with Password"):
  !unrar x -Y {Source_Path} {UnPack_Dest}
  print(output)
if(Action=="Tar"):
  !tar -cvf {Tar_Dest} {Source_Path}
  print(output)
if(Action=="UnTar"):
  !tar -xvf {Source_Path} -C {UnPack_Dest}
  print(output)
if(Action=="Tar.gz"):
  !tar -czvf {Targz_Dest} {Source_Path}
  print(output)
if(Action=="UnTar.gz"):
  !tar -xzvf {Source_Path} -C {UnPack_Dest}
  print(output)
if(Action=="Tar.bz2"):
  !tar -cjvf {Tarbz_Dest} {Source_Path}
  print(output)
if(Action=="UnTar.bz2"):
  !tar -xjvf {Source_Path} -C {UnPack_Dest}
  print(output)
if(Action=="Extract 7z"):
  %cd {sz_Dest}
  !7z x {Source_Path}

## **Advanced Task: Run Commands**

**You can run any supported Command below after downloading its setup from above!<br>List of Commands:**
```
# Putting '!' before any command is mandatory!
---
!rclone config - Enter an interactive configuration session.
!rclone copy - Copy files from source to dest, skipping already copied.
!rclone sync - Make source and dest identical, modifying destination only.
!rclone move - Move files from source to dest.
!rclone delete - Remove the contents of path.
!rclone purge - Remove the path and all of its contents.
!rclone mkdir - Make the path if it doesn't already exist.
!rclone rmdir - Remove the path.
!rclone rmdirs - Remove any empty directories under the path.
!rclone check - Check if the files in the source and destination match.
!rclone ls - List all the objects in the path with size and path.
!rclone lsd - List all directories/containers/buckets in the path.
!rclone lsl - List all the objects in the path with size, modification time and path.
!rclone md5sum - Produce an md5sum file for all the objects in the path.
!rclone sha1sum - Produce a sha1sum file for all the objects in the path.
!rclone size - Return the total size and number of objects in remote:path.
!rclone version - Show the version number.
!rclone cleanup - Clean up the remote if possible.
!rclone dedupe - Interactively find duplicate files and delete/rename them.
!rclone authorize - Remote authorization.
!rclone cat - Concatenate any files and send them to stdout.
!rclone copyto - Copy files from source to dest, skipping already copied.
!rclone genautocomplete - Output shell completion scripts for rclone.
!rclone gendocs - Output markdown docs for rclone to the directory supplied.
!rclone listremotes - List all the remotes in the config file.
!rclone mount - Mount the remote as a mountpoint.
!rclone moveto - Move file or directory from source to dest.
!rclone obscure - Obscure password for use in the rclone.conf
!rclone cryptcheck - Check the integrity of an encrypted remote.
!rclone about - Get quota information from the remote.
---
!yt-dlp [OPTIONS] [URL-1 URL-2 URL-3]
---
!aria2c <torrent/magnet/url>
```

**Write your Code Below:**

**Code 1**

In [ ]:
  #<--Write Code before #

**Code 2**

In [ ]:
  #<--Write Code before #

**Powered By:<br>**
```
Python: v3.10
Rclone: v1.58.1
yt-dlp: v2022.05.18
aria2: v1.36.0
LibTorrent: v1.0.5
Installer: v2.2.9 (R-1581)
NoteBook: v3.0.5
```